In [1]:
import numpy as np
import random as rand
from tqdm import tqdm

Want to sample from three variables:
1. $\lambda$
2. $\tau$
3. $\beta$

-> these values are then averaged to get the Monte-Carlo estimator
-> first: what is density of $\beta$?

In [2]:
# import data from DNN training
extracted_coefficients_directory = '../../../../data/commaai/extracted_coefficients/20201021_unrestr_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
z = np.load(z_path)
tBB = B_zeta.T.dot(B_zeta)

theta_prior = 2.5

In [3]:
B_zeta.shape

(355543, 10)

In [4]:
beta = np.repeat(0,10)
# number of samples we want to produce
M = 4000 + 1
L = 80

# number of parameters of theta
q = B_zeta.shape[1] + 1
p = B_zeta.shape[1]

n = B_zeta.shape[0]

# start values for beta, lambda, tau
# beta is taken from trained network and other two
# are initialized randomly
theta_m_1 = np.append(np.zeros(10), np.random.rand(1,))

# stepsize
epsilon = 0.00015

r_m = np.zeros(theta_m_1.shape[0])

tau_start = theta_m_1[p]

In [5]:
W = np.array([B_zeta[i,:].dot(B_zeta[i,:]) for i in range(0, n)])
S = np.sqrt(1/(1 + W*tau_start))
S2 = S**2

 need to write functions for
- log-density of $\log \lambda_j^2$
- gradient of log-density of $\log \lambda_j^2$

- log-density of $\log \tau$
- gradient of $\log \tau$

- log-density of $\beta$
- gradient of $\beta$

In [6]:
def delta_beta(z, u, B, S, beta, tBB):
    return(B.T.dot(z*(1/S)) - beta.T.dot(tBB) - beta/np.exp(u))

def delta_tau(u, B, S2, dS2, z, beta, theta_prior, betaBt):
    p = B.shape[1]
    return(- 0.5*np.sum(dS2/S2) 
           - 0.5*np.sum((z**2)*(-dS2/(S2**2)))
           + np.sum(betaBt*((-0.5*dS2/(S2**1.5))*z)) 
           - (0.5*p - 0.5) 
           + 0.5*(beta.T.dot(beta))/np.exp(u) 
           - 0.5*(np.exp(u)/theta_prior)**0.5 )

def compdS(tau2, W):
    tildeW = tau2*W
    S2 = 1/(1+tildeW)
    dS2 = -tildeW/((1+tildeW)**2)
    ddS2 = -tildeW/((1+tildeW)**2) + 2*(tildeW**2)/((1+tildeW)**3)
    
    S = np.sqrt(S2)
    return(S2, dS2, ddS2, S)

def Delta_theta(vartheta_t, B, n, z, p, tBB, betaBt, theta, W, theta_prior):
    vartheta_new = vartheta_t.copy()
    beta_t = vartheta_new[0:p].reshape(p,)
    u = vartheta_new[p]

    S2, dS2, ddS2, S = compdS(np.exp(u), W)
    
    # Gradient w.r.t. beta
    grad_beta = delta_beta(z, u, B, S, beta_t, tBB)
    # Gradient w.r.t. tau
    grad_tau = delta_tau(u, B, S2, dS2, z, beta_t, theta_prior, betaBt)
    
    return(np.append(grad_beta, grad_tau))

def log_density(z, u,  beta, B, p,  n, S, S2, tBB, theta, betaBt):   
    term1 = - 0.5*np.sum(np.log(S2))
    term2 =  - 0.5*z.dot((1/S2)*z)
    term3 = + betaBt.dot(z*(1/S)) 
    term4 = - 0.5*beta.T.dot(tBB).dot(beta)
    term5 =  - 0.5/np.exp(u)*np.sum(beta**2)
    term6 =  - 0.5*u*(p-1)  - np.sqrt(np.exp(u)/theta)
    return (term1 + term2 + term3 +term4 + term5 + term6  )



In [7]:
def Leapfrog(theta, r, epsilon, n, z, p, B, tBB, betaBt, beta, W, theta_prior):
    
    # compute gradient with theta
    Delta_theta_t = Delta_theta(theta, B, n, z, p, tBB, betaBt, theta, W, theta_prior)

    # update momentum
    r_tilde = r + (epsilon/2)*Delta_theta_t

    # update theta
    theta_tilde = (theta + epsilon*r_tilde).reshape(p + 1,)
    beta_t = theta_tilde[0:p]
    betaBt_t = beta_t.dot(B_zeta.T)
    
    # compute updated gradient
    Delta_theta_tilde = Delta_theta(theta_tilde, B, n, z, p, tBB, betaBt_t, theta, W, theta_prior)
    
    # update momentum again
    r_tilde = r_tilde + (epsilon/2)*Delta_theta_tilde
    
    return(theta_tilde, r_tilde)

In [8]:
rand.seed(248367852945)
r0 = np.repeat(None, M)
theta_tilde = np.repeat(None, M)
r_tilde = np.repeat(None, M)
log_dens =  np.repeat(None, M)
alpha = np.repeat(None, M)
theta_m_1 = np.repeat(None, M)
r_m = np.repeat(None, M)
theta_m_1[0] = np.append(np.zeros(10), np.random.rand(1,))

acc = []
# loop over number of samples that we want to produce
theta_tilde[0] = np.zeros(11)
r_tilde[0] = np.zeros(11)
theta_m_1[1] = np.zeros(11)
all_thetas = []
theta_m_1[0] = np.zeros(11)
r_m[0] = np.random.multivariate_normal(np.zeros(q), np.identity(q), 1)
r_m[1] = np.random.multivariate_normal(np.zeros(q), np.identity(q), 1)

for m in tqdm(range(1, M - 1)):
    
    # Update S
    # draw momentum from normal distribution
    r0[m] = np.random.multivariate_normal(np.zeros(q), np.identity(q), 1)
    
    # set new parameters
    theta_tilde[m] = theta_m_1[m].reshape(11,) # current q
    r_tilde[m] = r0[m]
    betaBt = theta_tilde[m][0:p].dot(B_zeta.T)
    
    # generate proposal through L leapfrog updates 
    for i in range(0,L):
        theta_tilde[m], r_tilde[m] = Leapfrog(theta_tilde[m], r_tilde[m], epsilon, n, z, p, B_zeta, tBB, betaBt, beta, W, theta_prior)
    
    S2, dS2, ddS2, S = compdS(np.exp(theta_tilde[m][p]), W)
    betaBt = theta_tilde[m][0:p].dot(B_zeta.T)
    
    # rtilde = proposed p
    # theta_tilde[m] = proposed u 
    # theta_m_1[m] = current u
    # theta_m_1[m] = current u
    # probability that proposal is accepted
    # current U
    log_dens[m] = log_density(z, theta_tilde[m][p],  theta_tilde[m][0:p], B_zeta, p,  n, S, S2, tBB, theta_prior, betaBt)
    proposed_u = log_density(z, theta_m_1[m][p],  theta_m_1[m][0:p], B_zeta, p,  n, S, S2, tBB, theta_prior, betaBt)
    current_K = r_tilde[m].dot(r_tilde[m].T)*0.5
    proposed_K = r_m[m].dot(r_m[m].T)*0.5
    p_accept = log_dens[m] -  proposed_u + current_K - proposed_K
    #decision = np.exp(log_dens[m] - r_tilde[m].dot(r_tilde[m].T)*1/2*(log_dens[m] - r0[m].dot(r0[m].T)*1/2))
    alpha[m] = np.exp(min([np.log(1), p_accept]))
    
    #decision = np.random.uniform(0,1,1)
    if np.random.randn() <= alpha[m]:
        theta_m_1[m + 1] = theta_tilde[m]
        r_m[m + 1] = - r_tilde[m]
        acc.append(1)
        all_thetas.append(np.array(theta_m_1[m + 1]))
    else:
        theta_m_1[m + 1] = theta_tilde[m - 1]
        r_m[m + 1] = - r_tilde[m - 1]
        acc.append(0)
        all_thetas.append(np.array(theta_m_1[m + 1]))
    if ((m % 1000 == 0) & (m > 1)):
        print(np.mean(acc[(m - 500):]))
        
    #if m < tune:
    #    epsilon, _ = step_size_tuning.update(alpha[m])
    #    print(epsilon)
    #elif m == tune:
    #    _, epsilon = step_size_tuning.update(alpha[m])
np.save('../../../../data/commaai/mcmc/unfiltered_gaussian_resampled/Ridge/vartheta_new.npy', np.array(all_thetas))    

 25%|██▌       | 1000/3999 [4:16:49<13:26:19, 16.13s/it]

0.672


 50%|█████     | 2000/3999 [8:32:41<8:52:53, 15.99s/it] 

0.676


 75%|███████▌  | 3000/3999 [12:03:00<3:11:53, 11.52s/it]

0.646


100%|██████████| 3999/3999 [15:05:38<00:00, 13.59s/it]  


In [ ]:
np.mean(acc)

In [ ]:
np.save('../../../../data/commaai/mcmc/unfiltered_gaussian_resampled/Ridge/all_thetas_L100_5000.npy', all_thetas)

In [ ]:
len(all_thetas)

In [ ]:
#np.save('../../../../../data/commaai/mcmc/filtered_gaussian_resampled/Ridge/all_thetas_L100.npy', all_thetas)

In [ ]:
all_thetas = np.load('../../../../../data/commaai/mcmc/filtered_gaussian_resampled/Ridge/all_thetas_try.npy')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(all_thetas)

In [ ]:
df = np.array(all_thetas)

In [ ]:
plt.plot(df[:100,0])
plt.ylim(-.1, .1)

In [ ]:
sd = []
for i in range(0,df.shape[1]):
    sd.append(np.std(df[0:200,i]))

In [ ]:
sd

In [ ]:
mu = []
for i in range(0,df.shape[1]):
    mu.append(np.mean(df[4000:,i]))

In [ ]:
B_ts = np.load('../../../../../data/commaai/va/unfiltered_gaussian_resampled/Ridge/B_ts.npy')
d_ts = np.load('../../../../../data/commaai/va/unfiltered_gaussian_resampled/Ridge/d_ts.npy')
B_ts = np.mean(B_ts[40000:,:,:], axis = 0)
d_ts = np.mean(d_ts[40000:,:,:], axis = 0)

In [ ]:
sd_va = np.sqrt(np.diag(B_ts.dot(B_ts.T) + d_ts**2))

In [ ]:
plt.scatter(sd_va[0:10], sd[0:10], color = 'b') #, col = ['b','b','b','b','b','b','b','b','b','b','r']
plt.scatter(sd_va[10], sd[10], color = 'r')
plt.xlabel('va')
plt.ylabel('mcmc')
x=np.linspace(0,0.05,1000) 
plt.plot(x,x,'k-') 

In [ ]:
mu_ts = np.load('../../../../../data/commaai/va/unfiltered_gaussian_resampled/Ridge/mu_ts.npy')
mu_ts = np.mean(mu_ts[40000:,:,:], axis = 0)

In [ ]:
plt.scatter(mu_ts[0:10], mu[0:10], color = 'b') #, col = ['b','b','b','b','b','b','b','b','b','b','r']
plt.scatter(np.exp(0.5*mu_ts[10]), np.exp(0.5*mu[10]), color = 'r')
plt.xlabel('va')
plt.ylabel('mcmc')
x=np.linspace(-0.5,0.5,1000) 
plt.plot(x,x,'k-') 

In [ ]:
sd

In [ ]:
mu_va = np.genfromtxt('../../../../../data/commaai/va/filtered_gaussian_resampled/Ridge/mu_t_va.csv', delimiter = ',')

In [ ]:
mu_va = np.load('../../../../../data/commaai/va/filtered_gaussian_resampled/Horseshoe/mu_ts.npy')

In [ ]:
np.mean(mu_va[20000:,:,:], axis = 0)

In [ ]:
-2.66744590e-02

In [ ]:
mu_va_mean = np.mean(mu_va[30000:,:], axis = 0)

In [ ]:
plt.scatter(mu_va_mean[0:10], np.array(mu[0:10])) #, col = ['b','b','b','b','b','b','b','b','b','b','r']
plt.xlabel('va')
plt.ylabel('mcmc')
x=np.linspace(0,0.05,1000) 
plt.plot(x,x,'k-') 

In [ ]:
import matplotlib.pyplot as plt
#axes = plt.gca()
#axes.plot(all_thetas)
plt.acorr(df[20000:,0])
#axes.set_ylim(-0.5, 0)

In [ ]:
theta_hmc = np.mean(all_thetas, axis = 0)
np.save('../../../../../data/commaai/mcmc/filtered_gaussian_resampled/Ridge/all_thetas2.npy', all_thetas)

In [ ]:
# comparison with VA

In [ ]:
vartheta_va = np.load('../../../../../data/commaai/va/filtered_gaussian_resampled/Ridge/vartheta.npy')#np.genfromtxt('../../../data/commaai/va/filtered_gaussian_resampled/Ridge/vartheta.csv', delimiter=",")

In [ ]:
theta_va = np.mean(vartheta_va[int(0.9*50000):50000, :], axis = 0)

In [ ]:
0.9*50000

In [ ]:
plt.scatter(theta_hmc[0:10], theta_va[0:10])
ident = [-1, 1.0]
plt.plot(ident,ident)

In [ ]:
beta_va = theta_hmc[0:10]

In [ ]:
beta_va.shape

In [ ]:
all_thetas = np.load('../../../../../data/commaai/mcmc/filtered_gaussian_resampled/Horseshoe/all_thetas_L100.npy')

In [ ]:
plt.plot(all_thetas)